<a href="https://colab.research.google.com/github/Aids42/Data-Analysis/blob/master/Melburn_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка данных к работе

Ссылка на датасет 

https://drive.google.com/file/d/1vSzvhBrJ7qBJFCaWRG3yP5_IuArIV6gF/view?usp=share_link

In [50]:
! gdown 1vSzvhBrJ7qBJFCaWRG3yP5_IuArIV6gF


Downloading...
From: https://drive.google.com/uc?id=1vSzvhBrJ7qBJFCaWRG3yP5_IuArIV6gF
To: /content/leopard-challenge-regression.zip
100% 1.00M/1.00M [00:00<00:00, 150MB/s]


In [51]:
! unzip -o /content/leopard-challenge-regression.zip -d /content/drive/MyDrive/DS_train/leopard_regression

Archive:  /content/leopard-challenge-regression.zip
  inflating: /content/drive/MyDrive/DS_train/leopard_regression/sample_submission.csv  
  inflating: /content/drive/MyDrive/DS_train/leopard_regression/test.csv  
  inflating: /content/drive/MyDrive/DS_train/leopard_regression/train.csv  


In [52]:
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Предпросмотр данных

In [53]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV

from catboost import CatBoostRegressor

from catboost import cv

In [54]:
data = pd.read_csv('/content/drive/MyDrive/DS_train/leopard_regression/train.csv')

In [55]:
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,id
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0,34302
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0,33247
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0,31886
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra City Council,-37.7969,144.9969,Northern Metropolitan,4019.0,18999
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra City Council,-37.8072,144.9941,Northern Metropolitan,4019.0,16809


Описание столбцов (параметров объекта):  
Suburb – Район  
Address – адрес  
Rooms – количество комнат

Type – тип объекта  
+ br - bedroom(s);  
+ h - house,cottage,villa, semi,terrace;  
+ u - unit, duplex;  
+ t - townhouse;  
+ dev site - development site;  
+ o res - other residential. 

Price – цена объекта  

Method – метод продажи объекта:  
+ S - property sold;
+ SP - property sold prior;
+ PI - property passed in;
+ PN - sold prior not disclosed;
+ SN - sold not disclosed;
+ NB - no bid;
+ VB - vendor bid;
+ W - withdrawn prior to auction;
+ SA - sold after auction;
+ SS - sold after auction price not disclosed.
+ N/A - price or highest bid not available.

SellerG – имя риэлтора  
Date – дата продажи объекта  
Distance – расстояние до центрального района в километрах  
Postcode – почтовый индекс  
Bedrooms2 – количество спален   
Bathroom – количество ванных комнат   
Car – количество парковочных мест  
Landsize – площадь прилегающей территории   
BuildingArea – площадь самого объекта  
YearBuilt – год постройки  
CouncilArea – округ  
Lattitude – широта  
Longtitude – долгота  
Regionname – еще одно административное деление на подобие округа  
Propertycount – не совсем ясно что, возможно количество объектов в округе  
id – id объекта  


In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18373 entries, 0 to 18372
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         18373 non-null  object 
 1   Address        18373 non-null  object 
 2   Rooms          18373 non-null  int64  
 3   Type           18373 non-null  object 
 4   Price          18373 non-null  float64
 5   Method         18373 non-null  object 
 6   SellerG        18373 non-null  object 
 7   Date           18373 non-null  object 
 8   Distance       18373 non-null  float64
 9   Postcode       18373 non-null  float64
 10  Bedroom2       14905 non-null  float64
 11  Bathroom       14903 non-null  float64
 12  Car            14798 non-null  float64
 13  Landsize       13579 non-null  float64
 14  BuildingArea   7750 non-null   float64
 15  YearBuilt      8944 non-null   float64
 16  CouncilArea    18372 non-null  object 
 17  Lattitude      15041 non-null  float64
 18  Longti

In [57]:
pd.options.display.max_columns=50  # увеличил количество отображаемых колонок

## Для начала уберу колонки содержащие пустые ячейки и попробую получить результат без них, затем добавлю

In [58]:
data[data['Address'].str.contains('Stafford') ] # вывел данные содержащие в адресе только Станффорд - потому что в этой строке очень много nan, больше чем в других (ну вот так, проверить)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,id
44,Abbotsford,60 Stafford St,3,h,1290000.0,S,Biggin,25/02/2017,2.5,3067.0,3.0,1.0,1.0,168.0,124.0,1950.0,Yarra City Council,-37.80070,144.99580,Northern Metropolitan,4019.0,9116
14406,Footscray,2/16 Stafford St,2,u,710000.0,S,Jas,15/07/2017,5.1,3011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21235
16801,Abbotsford,67 Stafford St,3,h,1000000.0,PI,Biggin,19/08/2017,3.0,3067.0,3.0,1.0,2.0,196.0,NaN,1900.0,Yarra City Council,-37.80094,144.99569,Northern Metropolitan,4019.0,3449


In [59]:
data = data.drop(labels=14406)

In [60]:
data.info() # так и есть удалился тот самый стаффорд 2/16

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18372 entries, 0 to 18372
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         18372 non-null  object 
 1   Address        18372 non-null  object 
 2   Rooms          18372 non-null  int64  
 3   Type           18372 non-null  object 
 4   Price          18372 non-null  float64
 5   Method         18372 non-null  object 
 6   SellerG        18372 non-null  object 
 7   Date           18372 non-null  object 
 8   Distance       18372 non-null  float64
 9   Postcode       18372 non-null  float64
 10  Bedroom2       14905 non-null  float64
 11  Bathroom       14903 non-null  float64
 12  Car            14798 non-null  float64
 13  Landsize       13579 non-null  float64
 14  BuildingArea   7750 non-null   float64
 15  YearBuilt      8944 non-null   float64
 16  CouncilArea    18372 non-null  object 
 17  Lattitude      15041 non-null  float64
 18  Longti

In [61]:
data = data.dropna(axis=1) # теперь удаляеи остальные nan

In [62]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18372 entries, 0 to 18372
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         18372 non-null  object 
 1   Address        18372 non-null  object 
 2   Rooms          18372 non-null  int64  
 3   Type           18372 non-null  object 
 4   Price          18372 non-null  float64
 5   Method         18372 non-null  object 
 6   SellerG        18372 non-null  object 
 7   Date           18372 non-null  object 
 8   Distance       18372 non-null  float64
 9   Postcode       18372 non-null  float64
 10  CouncilArea    18372 non-null  object 
 11  Regionname     18372 non-null  object 
 12  Propertycount  18372 non-null  float64
 13  id             18372 non-null  int64  
dtypes: float64(4), int64(2), object(8)
memory usage: 2.1+ MB


In [63]:
data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'CouncilArea', 'Regionname',
       'Propertycount', 'id'],
      dtype='object')

# Подготовка features

In [64]:
X = list(data.columns)
try:
  X.remove('Date') # дата продажи не интересна для предсказандия цены, так как не проданная квартира не будет иметь даты продажи
  #X.remove('id') # Ид объекта нам тоже не интересен
  X.remove('Price') # это цель
except:
  print('Уже нет этих полей')


y = ['Price'] # таргет

cat_features = ['Suburb',  # категориальные фичи для катбуста
                'Address', 
                'Rooms', 
                'Type', 
                'Method', 
                'SellerG',
                'Postcode', # по сути это категориальная
                'CouncilArea', 
                'Regionname' 
                ]


In [65]:
X # фичи

['Suburb',
 'Address',
 'Rooms',
 'Type',
 'Method',
 'SellerG',
 'Distance',
 'Postcode',
 'CouncilArea',
 'Regionname',
 'Propertycount',
 'id']

In [66]:
y # таргет

['Price']

In [67]:
# преобразуем Postcodе в string, иначе катбуст ругается

data['Postcode'] = data['Postcode'].astype(str)

In [68]:
model = CatBoostRegressor()


In [69]:
grid = GridSearchCV(model,
                    {
                           'random_seed': [42], 
                           'cat_features': [cat_features], 
                           'verbose': [100]
                     }, 
                    scoring='neg_mean_absolute_percentage_error')

In [70]:
grid.fit(data[X], data[y])

Learning rate set to 0.062604
0:	learn: 579378.3925521	total: 134ms	remaining: 2m 13s
100:	learn: 309423.3732593	total: 4.03s	remaining: 35.8s
200:	learn: 297581.9426487	total: 7.42s	remaining: 29.5s
300:	learn: 290169.9348205	total: 16.2s	remaining: 37.6s
400:	learn: 284225.0278720	total: 23.4s	remaining: 34.9s
500:	learn: 278425.4497909	total: 29.3s	remaining: 29.2s
600:	learn: 273435.0839175	total: 32.7s	remaining: 21.7s
700:	learn: 268697.3715583	total: 35.8s	remaining: 15.3s
800:	learn: 264871.4400278	total: 39s	remaining: 9.68s
900:	learn: 260948.3646889	total: 44.3s	remaining: 4.86s
999:	learn: 257114.8787106	total: 47.4s	remaining: 0us
Learning rate set to 0.062604
0:	learn: 598214.8584253	total: 35.3ms	remaining: 35.2s
100:	learn: 316750.1389606	total: 2.88s	remaining: 25.7s
200:	learn: 304746.9448679	total: 5.84s	remaining: 23.2s
300:	learn: 296313.0013369	total: 11.2s	remaining: 25.9s
400:	learn: 291295.2954803	total: 14.2s	remaining: 21.2s
500:	learn: 285483.3961501	total: 

GridSearchCV(estimator=<catboost.core.CatBoostRegressor object at 0x7ff9cc32dcd0>,
             param_grid={'cat_features': [['Suburb', 'Address', 'Rooms', 'Type',
                                           'Method', 'SellerG', 'Postcode',
                                           'CouncilArea', 'Regionname']],
                         'random_seed': [42], 'verbose': [100]},
             scoring='neg_mean_absolute_percentage_error')

In [71]:
grid.best_score_

-0.19788254370305747

In [72]:
data_test = pd.read_csv('/content/drive/MyDrive/DS_train/leopard_regression/test.csv')

In [73]:
data_test2 = data_test


In [74]:
data_test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8818 entries, 0 to 8817
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         8818 non-null   object 
 1   Address        8818 non-null   object 
 2   Rooms          8818 non-null   int64  
 3   Type           8818 non-null   object 
 4   Method         8818 non-null   object 
 5   SellerG        8818 non-null   object 
 6   Date           8818 non-null   object 
 7   Distance       8817 non-null   float64
 8   Postcode       8817 non-null   float64
 9   Bedroom2       5858 non-null   float64
 10  Bathroom       5854 non-null   float64
 11  Car            5582 non-null   float64
 12  Landsize       4365 non-null   float64
 13  BuildingArea   2882 non-null   float64
 14  YearBuilt      3112 non-null   float64
 15  CouncilArea    8816 non-null   object 
 16  Lattitude      5908 non-null   float64
 17  Longtitude     5908 non-null   float64
 18  Regionna

In [75]:
data_test2[(data_test2['Distance'].isna() | data_test2['CouncilArea'].isna() | data_test2['Regionname'].isna() | data_test2['Propertycount'].isna() )]

,Suburb,Address,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,id
2634,Camberwell,2/3 Kingsley St,2,h,VB,Jellis,11/11/2017,7.7,3124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15921
4628,Fawkner Lot,1/3 Brian St,3,h,SP,Brad,6/01/2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33262


In [76]:
data_test2.fillna(0, inplace=True)
data_test2['Postcode'] = data_test2['Postcode'].astype(str)


In [77]:
data_test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8818 entries, 0 to 8817
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         8818 non-null   object 
 1   Address        8818 non-null   object 
 2   Rooms          8818 non-null   int64  
 3   Type           8818 non-null   object 
 4   Method         8818 non-null   object 
 5   SellerG        8818 non-null   object 
 6   Date           8818 non-null   object 
 7   Distance       8818 non-null   float64
 8   Postcode       8818 non-null   object 
 9   Bedroom2       8818 non-null   float64
 10  Bathroom       8818 non-null   float64
 11  Car            8818 non-null   float64
 12  Landsize       8818 non-null   float64
 13  BuildingArea   8818 non-null   float64
 14  YearBuilt      8818 non-null   float64
 15  CouncilArea    8818 non-null   object 
 16  Lattitude      8818 non-null   float64
 17  Longtitude     8818 non-null   float64
 18  Regionna

In [78]:
data_out = grid.predict(data_test2[X])

In [93]:
data_out

array([ 915284.28911414, 1501775.84667946, 1448946.04164472, ...,
        637276.07002594, 1045325.68700179,  760612.85929821])

In [96]:
data_test2['id'].values

array([30130, 18311, 18934, ...,  2209, 23872,  8572])

In [101]:
d = pd.DataFrame(data={'id': data_test2['id'].values, 'Price': data_out})

In [105]:
d.head()

,id,Price
0,30130,9.152843e+05
1,18311,1.501776e+06
2,18934,1.448946e+06
3,1824,8.487015e+05
4,31339,8.731584e+05


In [106]:
d.to_csv('/content/drive/MyDrive/DS_train/leopard_regression/submission.csv', index=False)  